# <center>World Map Geovisualization using Folium and Covid Data

### Import Libraries

In [1]:
# importing Libraries

import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

### Access Data via API

In [2]:
connect = http.client.HTTPSConnection("api.covid19api.com") #connecting to the api
payload = ''
connect.request("GET", "/summary", payload, headers ={})
response = connect.getresponse()
data = response.read().decode('UTF-8')

### Convert Data to JSON

In [3]:
covid_data = json.loads(data)

### Normalize Data 

In [4]:
pd.json_normalize(covid_data['Countries'])

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,77,39693,2,1472,0,33058,2020-10-10T09:53:55Z
1,Albania,AL,albania,167,15066,2,413,89,9304,2020-10-10T09:53:55Z
2,Algeria,DZ,algeria,146,52804,6,1789,109,37067,2020-10-10T09:53:55Z
3,Andorra,AD,andorra,128,2696,1,55,99,1814,2020-10-10T09:53:55Z
4,Angola,AO,angola,73,6031,4,212,50,2685,2020-10-10T09:53:55Z
...,...,...,...,...,...,...,...,...,...,...
183,Viet Nam,VN,vietnam,5,1105,0,35,1,1024,2020-10-10T09:53:55Z
184,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-10-10T09:53:55Z
185,Yemen,YE,yemen,1,2051,0,593,0,1329,2020-10-10T09:53:55Z
186,Zambia,ZM,zambia,38,15339,1,336,74,14439,2020-10-10T09:53:55Z


### Convert to Pandas DataFrame

In [5]:
df = pd.DataFrame(covid_data['Countries'])
df.head(10)

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,77,39693,2,1472,0,33058,2020-10-10T09:53:55Z,{}
1,Albania,AL,albania,167,15066,2,413,89,9304,2020-10-10T09:53:55Z,{}
2,Algeria,DZ,algeria,146,52804,6,1789,109,37067,2020-10-10T09:53:55Z,{}
3,Andorra,AD,andorra,128,2696,1,55,99,1814,2020-10-10T09:53:55Z,{}
4,Angola,AO,angola,73,6031,4,212,50,2685,2020-10-10T09:53:55Z,{}
5,Antigua and Barbuda,AG,antigua-and-barbuda,0,111,0,3,0,97,2020-10-10T09:53:55Z,{}
6,Argentina,AR,argentina,15099,871468,515,23225,12297,697141,2020-10-10T09:53:55Z,{}
7,Armenia,AM,armenia,614,55087,6,1010,216,45528,2020-10-10T09:53:55Z,{}
8,Australia,AU,australia,18,27244,0,897,16,24987,2020-10-10T09:53:55Z,{}
9,Austria,AT,austria,1131,53188,4,842,750,42039,2020-10-10T09:53:55Z,{}


### Drop Unnecessary Columns

In [6]:
cdf = df.drop(columns=['CountryCode', 'Slug', 'Date', 'Premium'], axis=1)
cdf.head(10)

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,77,39693,2,1472,0,33058
1,Albania,167,15066,2,413,89,9304
2,Algeria,146,52804,6,1789,109,37067
3,Andorra,128,2696,1,55,99,1814
4,Angola,73,6031,4,212,50,2685
5,Antigua and Barbuda,0,111,0,3,0,97
6,Argentina,15099,871468,515,23225,12297,697141
7,Armenia,614,55087,6,1010,216,45528
8,Australia,18,27244,0,897,16,24987
9,Austria,1131,53188,4,842,750,42039


### Generate Base Map

In [7]:
base_map = folium.Map(tiles='Stamen Terrain', min_zoom=1.5)
base_map

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [8]:
url= 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

### Generate Choropleth Map Layer

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom = 2,
    name='COVID-19',
    data= cdf,
    columns=['Country', 'TotalConfirmed'], 
    key_on='feature.properties.name', 
    fill_color='OrRd',
    nan_fill_color='black',
    legend_name='Total Confirmed Cases'
).add_to(base_map)
base_map

### Generate Circular Markers

In [11]:
cdf.update(cdf['TotalConfirmed'].map('Total Confirmed: {}'.format))
cdf.update(cdf['TotalRecovered'].map('Total Recovered:{}'.format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [12]:
coordinates = pd.read_csv('country-coordinates-world.csv')
coordinates.head()

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra


In [13]:
# inner merging of two dataset
covid_df = pd.merge(cdf, coordinates, on='Country')
covid_df.head()


,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,latitude,longitude
0,Afghanistan,77,Total Confirmed: 39693,2,1472,0,Total Recovered:33058,33.939110,67.709953
1,Albania,167,Total Confirmed: 15066,2,413,89,Total Recovered:9304,41.153332,20.168331
2,Algeria,146,Total Confirmed: 52804,6,1789,109,Total Recovered:37067,28.033886,1.659626
3,Andorra,128,Total Confirmed: 2696,1,55,99,Total Recovered:1814,42.546245,1.601554
4,Angola,73,Total Confirmed: 6031,4,212,50,Total Recovered:2685,-11.202692,17.873887


In [20]:
# function to generation markers based on coordinates

def plot_marker(coordinate):
    folium.CircleMarker(location=[coordinate.latitude, coordinate.longitude],
    radius=5,
    weight = 2,
    popup=[coordinate.Country, coordinate.TotalConfirmed, coordinate.TotalRecovered],
    fill_color='#000000').add_to(base_map)

In [21]:
covid_df.apply(plot_marker, axis=1)
base_map.fit_bounds(base_map.get_bounds())
base_map

### Generate Base Map

### Generate Heat Map Layer

### Generate Circular Marker